In [1]:
#setup the environment for the NN, depending on if it to be run on Google Colab, or a local device (with or without a CUDA-enable GPU)

import os
import csv
import numpy as np
from tqdm import tqdm
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import time
#set up environment
REBUILD_DATA = True # set to true to one once, then back to false unless you want to change something in your training data.
GOOGLE_COLAB = False #set to true if running in google colab, false other wise

if torch.cuda.is_available():
    device = torch.device("cuda:0")  # you can continue going on here, like cuda:1 cuda:2....etc. 
    print("Running on the GPU")
else:
    device = torch.device("cpu")
    print("Running on the CPU")

if GOOGLE_COLAB:
    from google.colab import drive
    drive.mount('/content/gdrive')
    training_data_filename = "/content/gdrive/My Drive/Colab Notebooks/LoL-Predictor/datasets/training_data.csv" #for running on google colab notebooks
    model_log_filename = "/content/gdrive/My Drive/Colab Notebooks/LoL-Predictor/logs/model.log"
    api_key_location = "/content/gdrive/My Drive/Colab Notebooks/LoL-Predictor/api_key.txt"
    numpy_training_data = "/content/gdrive/My Drive/Colab Notebooks/LoL-Predictor/datasets/training_data.npy"
else:
    training_data_filename = "C:\\Users\\James Ting\\OneDrive - McGill University\\Personal\\Personal Projects\\LoL-Predictor\\datasets\\training_data.csv"
    model_log_filename = "C:\\Users\\James Ting\\OneDrive - McGill University\\Personal\\Personal Projects\\LoL-Predictor\\logs\\model.log"
    api_key_location = 'C:\\Users\\James Ting\\OneDrive - McGill University\\Personal\\Personal Projects\\LoL-Predictor\\api_key.txt'
    numpy_training_data = "C:\\Users\\James Ting\\OneDrive - McGill University\\Personal\\Personal Projects\\LoL-Predictor\\datasets\\training_data.npy"

f = open(api_key_location,'r')
api_key = f.readline()
f.close()

region = 'na1'



Running on the CPU


In [2]:

class DataSetReader():
    LABELS = {'blue_win':0,
              'red_win':1}
    training_data = []
    blue_win_count = 0
    red_win_count = 0
    training_data_location = training_data_filename
    exclude_columns = (0, 23, 25, 26, 29, 98, 100, 101, 104, 173, 175, 176, 
                       179, 248, 250, 251, 254, 323, 325, 326, 329, 398, 400, 
                       401, 404, 473, 475, 476, 479, 548, 550, 551, 554, 623, 
                       625, 626, 629, 698, 700, 701, 704)


    def read_training_data(self):
      with open(self.training_data_location,'r') as f:
        next(f) #skip the line with the column names
        for row in f:
          matchData = self.read_and_exclude_row(row,self.exclude_columns)
          result = None
          if(matchData[-1] == 1): #red win, blue loos
            result = np.eye(2)[1] #[1,0]
            self.blue_win_count += 1
          elif (matchData[-1] == 0): #red loss, blue win
            result = np.eye(2)[0]
            self.red_win_count += 1
          else:
            raise TypeError
          self.training_data.append([np.array(matchData[:-2]),result])
      
      
      f.close()
      np.random.shuffle(self.training_data)
      np.save(numpy_training_data, self.training_data)
      print('Blue Win Count:',self.blue_win_count)
      print('Red Win Count:', self.red_win_count)
          
          

    def read_and_exclude_row(self,row,exclude_column_indices):
        row_list = row.split(',')
        cleaned_row = []
        append = cleaned_row.append
        for index,value in enumerate(row_list):
            if index not in exclude_column_indices:
              if value == 'FALSE' or value == 'False':
                append(0)
              elif value == 'TRUE' or value == 'True':
                append(1)
              else:
                append(float(value))
                  
               
        return cleaned_row

if REBUILD_DATA:
  data_reader = DataSetReader()
  data_reader.read_training_data()

Blue Win Count: 1332
Red Win Count: 1291
